### Setup

In [1]:
import pandas as pd
import ast

In [2]:
local = pd.read_csv('datasets/territory_codes_local.csv')
foreign = pd.read_csv('datasets/territory_codes_foreign.csv')
codes = (
    pd.concat([local, foreign])
    .rename(columns={'name': 'freguesia'})
    .reset_index(drop=True)
)

results = False

### Methods

In [16]:
def getResultsTable(file):
    resultsParty = file.loc['resultsParty', 'currentResults']

    results = (
        pd.DataFrame(ast.literal_eval(resultsParty))
        .drop(['absoluteMajority', 'constituenctyCounter', 'imageKey', 'mandates', 'presidents'], axis=1)
        .set_index('acronym')
    )

    return results

def getMetadata(file):
    metadata = ( 
        # *idea: check displayMessage and whatever else for error messages to print

        file.drop(['currentResults', 'displayMessage', 'foreignCounterMessage', 'nationalCounterMessage'
                    , 'previousResults', 'refreshTimeout'], axis=1)
        .loc['availableMandates', :]
    )

    return metadata

def getResults(file):
    r = getResultsTable(file)
    metadata = getMetadata(file)
    r['tkey'] = metadata['territoryKey']
    r['parish'] = metadata['territoryFullName']

    r = r.reset_index().set_index(['parish', 'acronym'])

    return r

### Testing

In [4]:
file1 = ( #hardcoded for testing
    pd.read_csv('datasets\\Local\\Açores\\Angra do Heroísmo\\Altares.csv')
    .set_index('index')
)
file2 = (
    pd.read_csv('datasets\\Local\\Açores\\Angra do Heroísmo\\Angra (Nossa Senhora da Conceição).csv')
    .set_index('index')
)

#### Tentatively create MultiIndex from location


Scrapped due to being too heavy in computation

In [5]:
# cutCodes1  = codes.iloc[0:1000].drop(['territoryKey'], axis=1)
# cutCodes2  = codes.iloc[1001:2000].drop(['territoryKey'], axis=1)
# cutCodes3  = codes.iloc[2001:].drop(['territoryKey'], axis=1)

In [6]:
# index1 = pd.MultiIndex.from_product([cutCodes1.distrito, cutCodes1.concelho, cutCodes1.freguesia], names=['distrito', 'concelho', 'freguesia'])

In [7]:
# index2 = pd.MultiIndex.from_product([cutCodes2.distrito, cutCodes2.concelho, cutCodes2.freguesia], names=['distrito', 'concelho', 'freguesia'])

In [8]:
# index3 = pd.MultiIndex.from_product([cutCodes3.distrito, cutCodes3.concelho, cutCodes3.freguesia], names=['distrito', 'concelho', 'freguesia'])

In [9]:
# index = pd.MultiIndex.append(index1, index2)

In [10]:
# index = index.append(index3)

#### Testing cont

In [25]:
for i, row in codes.iterrows():
    loc = row['territoryKey'].split('-')[0].capitalize()
    file = (
        pd.read_csv('datasets\\' + loc + '\\' + row['distrito'] + '\\' + row['concelho'] + '\\' + row['freguesia'] + '.csv')
        .set_index('index')
    )
    results = pd.concat([results, getResults(file)])

In [31]:
results


percentage  \
parish                                             acronym                          
Altares                                            PPD/PSD.CDS-PP.PPM       43.17   
                                                   PS                       34.27   
                                                   CH                       12.15   
                                                   B.E.                      2.17   
                                                   IL                        1.95   
...                                                                           ...   
Restantes Postos Consulares Dos Países da Ásia ... JPP                       0.22   
                                                   MPT.A                     0.17   
                                                   ND                        0.13   
                                                   R.I.R.                    0.12   
                                                   E                         0.08   

                                                                       validVotesPercentage  \
parish                                             acronym                                    
Altares                                            PPD/PSD.CDS-PP.PPM                 44.72   
                                                   PS                                 35.51   
                                                   CH                                 12.58   
                                                   B.E.                                2.25   
                                                   IL                                  2.02   
...                                                                                     ...   
Restantes Postos Consulares Dos Países da Ásia ... JPP                                 0.33   
                                                   MPT.A                               0.26   
                                                   ND                                  0.21   
                                                   R.I.R.                              0.18   
                                                   E                                   0.13   

                                                                       votes  \
parish                                             acronym                     
Altares                                            PPD/PSD.CDS-PP.PPM    199   
                                                   PS                    158   
                                                   CH                     56   
                                                   B.E.                   10   
                                                   IL                      9   
...                                                                      ...   
Restantes Postos Consulares Dos Países da Ásia ... JPP                    13   
                                                   MPT.A                  10   
                                                   ND                      8   
                                                   R.I.R.                  7   
                                                   E                       5   

                                                                                 tkey  
parish                                             acronym                             
Altares                                            PPD/PSD.CDS-PP.PPM    LOCAL-430101  
                                                   PS                    LOCAL-430101  
                                                   CH                    LOCAL-430101  
                                                   B.E.                  LOCAL-430101  
                                                   IL                    LOCAL-430101  
...                                                                     